In [1]:
# Built-in library
import asyncio
import json
import logging
import re
import warnings
from pathlib import Path
from pprint import pprint
from typing import Annotated, Any, Iterable, Literal, Optional, Union

# Standard imports
import nest_asyncio
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as pltife

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)

warnings.filterwarnings("ignore", category=UserWarning, module="numpy")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [4]:
go_up_from_current_directory(go_up=1)


from schemas import ModelEnum  # noqa: E402
from settings import refresh_settings  # noqa: E402
from utilities.client_utils import check_rate_limit  # noqa: E402

settings = refresh_settings()

/Users/neidu/Desktop/Projects/Personal/My_Projects/AI-Tutorials


<br><hr>

## [Outlines](https://dottxt-ai.github.io/outlines/latest/)

- Outlines is a Python library that allows you to use Large Language Model in a simple and robust way (with structured generation).
- It is built by .txt, and is already used in production by many companies.

### Features

- Make LLMs generate [valid JSON](https://dottxt-ai.github.io/outlines/latest/reference/generation/json/): No more invalid JSON outputs, 100% guaranteed

- JSON mode for [vLLM](https://dottxt-ai.github.io/outlines/latest/reference/serve/vllm/): Deploy a LLM service using Outlines' JSON structured generation and vLLM

- [Make LLMs follow a Regex](https://dottxt-ai.github.io/outlines/latest/reference/generation/regex/): Generate text that parses correctly 100% of the time

- [Powerful Prompt Templating](https://dottxt-ai.github.io/outlines/latest/reference/prompting/): Better manage your prompts' complexity with prompt templating


### Supported Models

- Openai, but the true power of Outlines is unleashed with Open Source models.

- All open source models available via the:
  - transformers
  - llama.cpp
  - exllama2
  - mlx-lm
  -  vllm models

### OpenAI Synchronous API

- `%%capture`: This is used to suppress the output of the cell.
- It was used here because of NumPy warnings.

- For scripts, use:

```sh
export PYTHONWARNINGS="ignore"
```

In [5]:
%%capture

from outlines import generate, models
from outlines.models.openai import OpenAIConfig
from pydantic import BaseModel, ConfigDict, Field, field_validator

In [6]:
from outlines import generate, models
from outlines.models.openai import OpenAIConfig
from pydantic import BaseModel, ConfigDict, Field, StringConstraints, field_validator

# %%capture disables intellisense.
# The packages are re-imported to enable intellisense.
config = OpenAIConfig(
    presence_penalty=1,
    frequency_penalty=1,
    temperature=0.0,
    top_p=0.95,
    seed=1,
)
# Using Ollama with any local model that supports the OpenAI API
model = models.openai(
    ModelEnum.QWEN_3p0_4B_LOCAL,
    api_key=settings.OLLAMA_API_KEY.get_secret_value(),
    base_url=settings.OLLAMA_URL,
    config=config,
)
model

OpenAIConfig(model=<ModelEnum.QWEN_3p0_4B_LOCAL: 'qwen3:4b-q4_K_M'>, frequency_penalty=1, logit_bias={}, max_tokens=None, n=1, presence_penalty=1, response_format=None, seed=1, stop=None, temperature=0.0, top_p=0.95, user='')

In [7]:
String = Annotated[
    str, StringConstraints(strip_whitespace=True, min_length=2, max_length=30)
]


class Person(BaseModel):
    first_name: String
    last_name: String = Field(default="Doe")
    age: int = Field(ge=5, le=100)


class Persons(BaseModel):
    persons: list[Person]


instructions: str = (
    "<inst>Given a text, extract the firstName, lastName and age. </inst>"
)
query: str = (
    "<text>James Bond, 42, is a debonair British secret agent. Armed with cool gadgets"
    "and charm, he undertakes dangerous global missions to stop villains and save the world in "
    "thrilling spy adventures. Prison break staring Michael Scofield was a massive hit! Michael "
    "was 34 years old when the show started and is now 53 years old. Lamine Yamal is just too "
    "good. It's almost impossible to believe that he's 17 years old.</text>"
)
generator = generate.json(model, Persons)
response = generator(f"{instructions}{query}")

console.log(response)

[13:33:46] Persons(                                                                                ]8;id=433161;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/2622450642.py\2622450642.py]8;;\:]8;id=292297;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/2622450642.py#29\29]8;;\
               persons=[                                                                                           
                   Person(first_name='James', last_name='Bond', age=42),                                           
                   Person(first_name='Michael', last_name='Scofield', age=53),                                     
                   Person(first_name='Lamine', last_name='Yamal', age=17)                                          
               ]                                                                                                   
           )                                                                                                       

In [8]:
query: str = (
    "There are two engineers working at GetIt AI. Kunle, 28 years old is "
    "a Python developer while Francis is a Golang guru at a ripe age of 32. Do "
    "not make any assumptions. Extract their information in a JSON format."
)
response = generator(f"{instructions}{query}")

console.log(response)

[13:39:46] Persons(                                                                                 ]8;id=965721;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/3667612818.py\3667612818.py]8;;\:]8;id=99507;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/3667612818.py#8\8]8;;\
               persons=[                                                                                           
                   Person(first_name='Kunle', last_name='Doe', age=28),                                            
                   Person(first_name='Francis', last_name='Doe', age=32)                                           
               ]                                                                                                   
           )                                                                                                       

In [9]:
# Track the API use
model.prompt_tokens, model.completion_tokens

(0, 0)

### OpenAI Asynchronous API

In [10]:
from openai import AsyncOpenAI

aclient: AsyncOpenAI = AsyncOpenAI(
    base_url=settings.OLLAMA_URL,
    api_key=settings.OLLAMA_API_KEY.get_secret_value(),
)
config = OpenAIConfig(
    model=ModelEnum.QWEN_3p0_4B_LOCAL,
    presence_penalty=1,
    frequency_penalty=1,
    temperature=0.0,
    top_p=0.95,
    seed=1,
)
async_model = models.openai(aclient, config)
async_model

OpenAIConfig(model=<ModelEnum.QWEN_3p0_4B_LOCAL: 'qwen3:4b-q4_K_M'>, frequency_penalty=1, logit_bias={}, max_tokens=None, n=1, presence_penalty=1, response_format=None, seed=1, stop=None, temperature=0.0, top_p=0.95, user='')

In [11]:
from typing import Type

from outlines.models.openai import OpenAI as OpenAIModel


async def generate_structured_output(
    model: OpenAIModel, prompt: str, response_model: Type[BaseModel]
) -> Type[BaseModel] | list[Type[BaseModel] | list]:
    """Generate structured output from a prompt using an OpenAI model.

    Parameters
    ----------
    model : OpenAIModel
        The OpenAI model instance to use for generation.
    prompt : str
        The input text prompt to generate structured output from.
    response_model : Type[BaseModel]
        The Pydantic model class defining the expected response structure.

    Returns
    -------
    Type[BaseModel] | list[Type[BaseModel] | list]
        The structured output matching the response_model schema.
        Can be either a single model instance or a list of model instances.
    """
    assert isinstance(prompt, str), "Prompt must be a string"

    generator = generate.json(model, response_model)
    return generator(prompt)

In [12]:
instructions: str = (
    "<inst>Given a text, extract the firstName, lastName and age. </inst>"
)
query: str = (
    "<text>James Bond, 42, is a debonair British secret agent. Armed with cool gadgets"
    "and charm, he undertakes dangerous global missions to stop villains and save the world in "
    "thrilling spy adventures. Prison break staring Michael Scofield was a massive hit! Michael "
    "was 34 years old when the show started and is now 53 years old. Lamine Yamal is just too "
    "good. It's almost impossible to believe that he's 17 years old.</text>"
)
prompt: str = f"{instructions}\n{query}"
response = await generate_structured_output(async_model, prompt, Persons)

console.log(response)

[14:03:27] Persons(                                                                                 ]8;id=737241;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/678038757.py\678038757.py]8;;\:]8;id=99084;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/678038757.py#14\14]8;;\
               persons=[                                                                                           
                   Person(first_name='James', last_name='Bond', age=42),                                           
                   Person(first_name='Michael', last_name='Scofield', age=53),                                     
                   Person(first_name='Lamine', last_name='Yamal', age=17)                                          
               ]                                                                                                   
           )                                                                                                       

## Prompt Templates

In [13]:
from outlines import Template

prompt = """Hello, {{ surname }}!"""
greetings = Template.from_string(prompt)

# Use keyword argument matching the template variable
# otherwise it will raise an error
prompt = greetings(surname="user")
prompt

'Hello, user!'

### Importing Prompts From Files

In [14]:
prompt_file: str = "./prompts/p1.jinja2"

greetings = Template.from_file(prompt_file)
prompt = greetings(name="user", question="How are you?")
console.log(prompt)

[14:03:37] Hello, user!                                                                             ]8;id=478697;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/1447805550.py\1447805550.py]8;;\:]8;id=484259;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/1447805550.py#5\5]8;;\
           How are you?                                                                                            
                                                                                                                   

In [15]:
prompt_file: str = "./prompts/p2.jinja2"
loaded_prompt = Template.from_file(prompt_file)

instructions: str = "Please answer the following question following the examples"
examples: list[dict[str, Any]] = [
    {"question": "2+2 = ?", "answer": "4"},
    {"question": "3+3 =? ", "answer": "6"},
]
question: str = "4+8 = ?"

prompt = loaded_prompt(instructions=instructions, examples=examples, question=question)
# console.log(prompt)
print(prompt)

<ins> Please answer the following question following the examples</ins> 

<ex>

Q: 2+2 = ?
A: 4

Q: 3+3 =? 
A: 6

</ex>

<qs>
Q: 4+8 = ?
A:
</qs>


In [16]:
class Answer(BaseModel):
    answer: float | int = Field(description="The answer to the question")


response = await generate_structured_output(async_model, prompt, Answer)

console.log(response)

[14:03:39] Answer(answer=12)                                                                        ]8;id=80942;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/3135518489.py\3135518489.py]8;;\:]8;id=314107;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/3135518489.py#7\7]8;;\

<br><br>

### JSON Response Format With Templates

- You can pass a **JSON schema** or a **Pydantic model** as a response model to the `generate.json` function.

In [17]:
class MyResponse(BaseModel):
    field1: int = Field(description="an int")
    field2: str


my_prompt = Template.from_string("""{{ response_model | schema }}""")
prompt = my_prompt(response_model=MyResponse)
print(prompt)

{
  "field1": "an int",
  "field2": "<field2>"
}


In [18]:
json_schema = MyResponse.model_json_schema()
json_schema

{'properties': {'field1': {'description': 'an int',
   'title': 'Field1',
   'type': 'integer'},
  'field2': {'title': 'Field2', 'type': 'string'}},
 'required': ['field1', 'field2'],
 'title': 'MyResponse',
 'type': 'object'}

In [19]:
my_prompt = Template.from_string("""{{ response_model | schema }}""")
prompt = my_prompt(response_model=json_schema)
print(prompt)

{
  "properties": {
    "field1": {
      "description": "an int",
      "title": "Field1",
      "type": "integer"
    },
    "field2": {
      "title": "Field2",
      "type": "string"
    }
  },
  "required": [
    "field1",
    "field2"
  ],
  "title": "MyResponse",
  "type": "object"
}


### Classification

- Using `outlines.generate.choices` with `greedy` sampler is not allowed with OpenAI.
- You can use the default sampler (`multinomial`) or use generate.json

In [20]:
customer_support: str = Template.from_string(
    """
    <inst>
        You are an sentiments analysis manager.

        Given a review from a user, determine if it's a POSITIVE or
        a NEGATIVE review.

        <ex>
            Review: "It was a great movie"
            Label: POSITIVE

            Review: "Such a waste of my time!"
            Label: NEGATIVE
        </ex>
    </inst>

    <task>
        What are the sentiments of the following reviews?
        {% for review in reviews %}
        Review: {{ review }}
        Label:
        {% endfor %}
    </task>
    """
)
prompt = customer_support(reviews=["It was a great movie", "Such a waste of my time!"])
print(prompt)

<inst>
    You are an sentiments analysis manager.

    Given a review from a user, determine if it's a POSITIVE or
    a NEGATIVE review.

    <ex>
        Review: "It was a great movie"
        Label: POSITIVE

        Review: "Such a waste of my time!"
        Label: NEGATIVE
    </ex>
</inst>

<task>
    What are the sentiments of the following reviews?
    Review: It was a great movie
    Label:
    Review: Such a waste of my time!
    Label:
</task>


In [21]:
class Sentiment(BaseModel):

    review: str = Field(description="The review")
    sentiment: Literal["POSITIVE", "NEGATIVE"] = Field(
        description="The sentiment of the text"
    )
    confidence: float = Field(description="The confidence score", ge=0, le=1)


class Sentiments(BaseModel):
    sentiments: list[Sentiment]


response = await generate_structured_output(
    async_model, prompt=prompt, response_model=Sentiments
)

console.log(response)

[14:03:47] Sentiments(                                                                             ]8;id=977953;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/4154712646.py\4154712646.py]8;;\:]8;id=847024;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_77427/4154712646.py#18\18]8;;\
               sentiments=[                                                                                        
                   Sentiment(                                                                                      
                       review='It was a great movie',                                                              
                       sentiment='POSITIVE',                                                                       
                       confidence=0.95                                                                             
                   ),                                                                                              
                   Sentiment(                                                                                      
                       review='Such a waste of my time!',                                                          
                       sentiment='NEGATIVE',                                                                       
                       confidence=0.98                                                                             
                   )                                                                                               
               ]                                                                                                   
           )                                                                                                       